In [1]:
#Import all the required libraries(install them if required.)
import json
from sseclient import SSEClient as EventSource
from collections import Counter
import datetime
import signal


#The url to fetch the Wikimedia data stream
url = ' https://stream.wikimedia.org/v2/stream/revision-create'
#These empty dictionary will be used to later to store the data minute wise.
domainReport1={}
users1={}
domainReport2={}
users2={}
domainReport3={}
users3={}
domainReport4={}
users4={}
domainReport5={}
users5={}
Fdomain ={}
Fusers = {}

#The below function is used to handle interrupts and terminate the program 
def signal_handler(signal, frame):
    global interrupted
    interrupted = True

signal.signal(signal.SIGINT, signal_handler)

#This function is used to fetch the datat from the API
def fetch_data():
    pageList = {}#to store the pages being updated
    userReport = {}#to store the users who made changes to the "en.wikipedia.org" domain
    timer = 0
    mins = 0 #to keep track of minutes
    time1 = datetime.datetime.now()
    global interrupted
    interrupted = False
    print("Generating reprot Please wait!..")
    #the fecting of data begins here
    for event in EventSource(url):
        if event.event == 'message':
            try:
                change = json.loads(event.data)            
            except ValueError:
                pass
            else:
                #as the data is received it is put into the required dictionary for further processing.
                if change["page_title"] not in pageList:
                    pageList[change["page_title"]] = change["meta"]["domain"]
                if change["meta"]["domain"] == "en.wikipedia.org":
                    if change["performer"]["user_is_bot"] == False:
                        count = change["rev_len"]
                        userReport[change["performer"]["user_text"]] = count #the user list
                time2 = datetime.datetime.now()
                t = time2- time1
                #This is to manage time. Currently the code is set to 5 sec interval
                if (t.seconds >= 5):
                    timer = timer + 1
                    mins = mins + 1
                    time1 = datetime.datetime.now()
                    print_func(pageList,userReport,timer,mins)
                    if(timer==5):
                        timer=0
                    #When an interrupt signal is received, terminate the program after completing the current cycle.
                    if interrupted:
                        print("\nYou have terminated the Program.Thank you!.")
                        break
    

# This function counts the number of pager per domain
def Convert(report):
    
    res = Counter(report.values())
    pageList = dict(res)    
    return pageList

#This function is used to sort the list based on the number of pages updated in the domain
def sorting(report):
    
    domainlist = list(report.items())
    domainlist.sort(key = lambda x: x[1])
    di = dict(domainlist)
    return di

# This function combines the reports in 5 minutes duration
def combine(report1,report2,report3,report4,report5):
    
    r1_counter = Counter(report1)
    r2_counter = Counter(report2)
    r3_counter = Counter(report3)
    r4_counter = Counter(report4)
    r5_counter = Counter(report5)
    add_counter = r1_counter + r2_counter +r3_counter +r4_counter +r5_counter
    Freport = dict(add_counter)
    return(Freport)

#This function will display the report!
def print_func(pageList,userReport,count,mins):
    global domainReport1,domainReport2,domainReport3,domainReport4,domainReport5,Fdomain
    global users1,users2,users3,users4,users5,Fusers
    
    if count == 1:
        domainReport1 = pageList
        domainReport1=Convert(domainReport1)
        users1 = userReport
    if count == 2:
        domainReport2 = pageList
        domainReport2=Convert(domainReport2)
        users2 = userReport
    if count == 3:
        domainReport3 = pageList
        domainReport3=Convert(domainReport3)
        users3 = userReport
    if count == 4:
        domainReport4 = pageList
        domainReport4=Convert(domainReport4)
        users4 = userReport
    if count == 5:
        domainReport5 = pageList
        domainReport5=Convert(domainReport5)
        users5 = userReport
        
    Fdomain = combine(domainReport1,domainReport2,domainReport3,domainReport4,domainReport5)
    Fusers = combine(users1,users2,users3,users4,users5)
    
    Fdomain = sorting(Fdomain)
    print("---------------------------------------------------------------------------------------------------------")
    print("\nReport for ",mins," minutes of data....") 
    print("\n######\t\t\tTotal number of Wikipedia Domains Updated:",len(Fdomain))
    for ele in Fdomain:
        print("\n",ele,":",Fdomain[ele]," pages updated.")
    print("\n######\t\t\tUsers who made changes to en.wikipedia.org are:\n")
    for ele in Fusers:
        print("\n",ele,":",Fusers[ele])

#This starts the program.   
fetch_data()

Generating reprot Please wait!..
---------------------------------------------------------------------------------------------------------

Report for  1  minutes of data....

######			Total number of Wikipedia Domains Updated: 12

 de.wikiversity.org : 1  pages updated.

 de.wikipedia.org : 1  pages updated.

 id.wikipedia.org : 1  pages updated.

 fr.wikipedia.org : 1  pages updated.

 meta.wikimedia.org : 1  pages updated.

 pl.wikipedia.org : 1  pages updated.

 ru.wikipedia.org : 2  pages updated.

 it.wikipedia.org : 2  pages updated.

 nl.wikipedia.org : 3  pages updated.

 en.wikipedia.org : 7  pages updated.

 www.wikidata.org : 14  pages updated.

 commons.wikimedia.org : 17  pages updated.

######			Users who made changes to en.wikipedia.org are:


 Tamart0290 : 74990

 158.62.69.219 : 56140

 ButterCashier : 125476

 110.136.13.21 : 150085

 185.55.105.228 : 1474
---------------------------------------------------------------------------------------------------------

Repor